In [35]:
import os
import shutil
import numpy as np
import tensorflow as tf
import datetime
print(tf.__version__)

1.15.0


In [2]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.14"                # TF version for CMLE to use

In [3]:
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [4]:
# 나중에 하이퍼파라미터로 쓸수 있는것.
HP_FC_EMBED_DIMENSION = 6
TRAIN_STEPS = 1000
EVAL_INTERVAL = 300


In [5]:
# Columns명 지정
CSV_COLUMNS = "weight_pounds,is_male,mother_age,plurality,gestation_weeks".split(',')
LABEL_COLUMN = "weight_pounds"
# Set default values for each CSV column
DEFAULTS = [[0.0], ["null"], [0.0], ["null"], [0.0]]

In [6]:
def add_engineered_features(features):
    features["dummy"] = features["mother_age"]
    
    return features

In [7]:
def read_dataset(filename_pattern, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(records = value_column, record_defaults = DEFAULTS)
            # Default feature
            features = dict(zip(CSV_COLUMNS, columns))
            # NEW: Add engineered features
            features = add_engineered_features(features)
            # Default label
            label = features.pop(LABEL_COLUMN)
            return features, label
    
        # Create list of files that match pattern
        file_list = tf.gfile.Glob(filename = filename_pattern)

        # Create dataset from file list
        dataset = (tf.data.TextLineDataset(filenames = file_list)  # Read text file
                     .map(map_func = decode_csv))  # Transform each elem by applying decode_csv fn

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(count = num_epochs).batch(batch_size = batch_size)
        return dataset
    return _input_fn

In [80]:
def get_categorical_indicator(name, values):
    return tf.feature_column.indicator_column(
        categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(key = name, vocabulary_list = values))

def get_feature_cols():
    # Vocabulary List
    voca_list_is_male = ["True","False","Unknown"]
    voca_list_plurality = ["Single(1)","Twins(2)","Triplets(3)","Quadruplets(4)","Quintuplets(5)","Multiple(2+)"]

    # Default Feature column
    fc_is_male = tf.feature_column.categorical_column_with_vocabulary_list(key="is_male", vocabulary_list=voca_list_is_male)
    fc_plurality = tf.feature_column.categorical_column_with_vocabulary_list(key="plurality", vocabulary_list=voca_list_plurality)
    fc_mother_age = tf.feature_column.numeric_column(key = "mother_age")
    fc_gestation_weeks = tf.feature_column.numeric_column(key = "gestation_weeks")
    # DNNRegressor모델인 경우에는, 아래의 함수를 써야 한다.
    # One-Hot Encoding을 해서 넣어야 하는거 같다.
    # fc_is_male   = get_categorical_indicator("is_male", voca_list_is_male)
    # fc_plurality = get_categorical_indicator("plurality", voca_list_plurality)

    
    # ADD Feature column
    fc_dummy = tf.feature_column.numeric_column(key = "dummy")

    # Bucketized columns
    fc_buckets_mother_age = tf.feature_column.bucketized_column(source_column = fc_mother_age, boundaries = np.arange(start = 15, stop = 45, step = 1).tolist())
    fc_buckets_gestation_weeks = tf.feature_column.bucketized_column(source_column = fc_gestation_weeks, boundaries = np.arange(start = 17, stop = 47, step = 1).tolist())
   
    # Embeded Feature columns
    crossed = tf.feature_column.crossed_column(keys=[fc_is_male,fc_plurality,fc_buckets_mother_age,fc_buckets_gestation_weeks], 
                                            hash_bucket_size = 20000)
    fc_embed = tf.feature_column.embedding_column(categorical_column = crossed, dimension = HP_FC_EMBED_DIMENSION)

    # Feature columns
    feature_columns = [fc_is_male,
                       fc_plurality,
                       fc_mother_age,
                       fc_gestation_weeks,
                       fc_dummy
                      ]
    
    # Sparse wide columns
    wide = [fc_is_male,fc_plurality,fc_buckets_mother_age,fc_buckets_gestation_weeks]
    
    #Deep colomns
    deep = [fc_mother_age,
            fc_gestation_weeks,
            fc_embed]
    
    return feature_columns, wide, deep

In [93]:
def serving_input_fn():
    feature_placeholders = {
        "is_male"        : tf.placeholder(dtype = tf.string,  shape = [None]),
        "mother_age"     : tf.placeholder(dtype = tf.float32, shape = [None]),
        "plurality"      : tf.placeholder(dtype = tf.string,  shape = [None]),
        "gestation_weeks": tf.placeholder(dtype = tf.float32, shape = [None])
    }
    
    features = add_engineered_features(feature_placeholders)
    
    # feature의 shape=(?,)이면, 아래 방법으로 차원을 증가시킨다.
    features = {
                key: tf.expand_dims(input = tensor, axis = -1)
                for key, tensor in feature_placeholders.items()
               }

    print(features["is_male"])
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

In [41]:
# BestExporter를 사용하게 되면, 아래 Function이 있어야 하며, compare_fn으로 사용한다.
def _accuracy_bigger(best_eval_result, current_eval_result):
    metric = 'accuracy'
    return best_eval_result[metric] < current_eval_result[metric]

In [75]:
def train_and_evaluate_DNNLinearCombinedRegressor(output_dir):
    feature_columns, wide, deep = get_feature_cols()
        
    run_config = tf.estimator.RunConfig(
        save_checkpoints_secs = EVAL_INTERVAL,
        keep_checkpoint_max = 3)
    
    estimator = tf.estimator.DNNLinearCombinedRegressor(
        model_dir = output_dir,
        linear_feature_columns = wide,
        dnn_feature_columns = deep,
        dnn_hidden_units = [128, 64, 32],
        config = run_config)
 
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset("train.csv", mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = TRAIN_STEPS)
    
    Last_exporter = tf.estimator.LatestExporter(name = "exporter", serving_input_receiver_fn = serving_input_fn)
    Final_exporter = tf.estimator.FinalExporter('./exporter', serving_input_receiver_fn=serving_input_fn)
    best_exporter = tf.estimator.BestExporter(
                  name="best_exporter",  # Saved models are exported under /export/best_exporter/
                  compare_fn=_accuracy_bigger,
                  event_file_pattern='eval_{}/*.tfevents.*'.format(datetime.datetime.utcnow().strftime("%H%M%S")),
                  serving_input_receiver_fn=serving_input_fn,
                  exports_to_keep=5 )
    exporters = [Final_exporter]
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset("eval.csv", mode = tf.estimator.ModeKeys.EVAL),
        steps = None,
        start_delay_secs = 60, # start evaluating after N seconds
        throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
        exporters = exporters)
    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [76]:
output_dir = "babyweight_trained_DNNLinearCombinedRegressor"
!rm -rf {output_dir}
train_and_evaluate_DNNLinearCombinedRegressor(output_dir)

INFO:tensorflow:Using config: {'_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4b88f69f28>, '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_device_fn': None, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_task_id': 0, '_is_chief': True, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_experimental_distribute': None, '_evaluation_master': '', '_save_checkpoints_secs': 300, '_master': '', '_session_creation_timeout_secs': 7200, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_experimental_max_worker_delay_secs': None, '_model_dir': 'babyweight_trained_DNNLinearCombinedRegressor', '_tf_random_seed': None, '_train_distribute': None, '_global_id_in_cluster': 0, '_task_type': 'worker', '_keep_checkpoint_max': 3, '_service': None, '_log_step_count_steps': 100}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow

In [90]:
def train_and_evaluate_LinearRegressor(output_dir):
    feature_columns, wide, deep = get_feature_cols()
    print('<<<<<{}>>>>><<<<<{}>>>>>'.format(type(feature_columns), feature_columns))
    
    run_config = tf.estimator.RunConfig(tf_random_seed = 1)
    
    estimator = tf.estimator.LinearRegressor(
        feature_columns = feature_columns,
        model_dir = output_dir,
        config = run_config)
    
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset("train.csv", mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = TRAIN_STEPS)
    
    Last_exporter = tf.estimator.LatestExporter(name = "exporter", serving_input_receiver_fn = serving_input_fn)
    Final_exporter = tf.estimator.FinalExporter('./exporter', serving_input_receiver_fn=serving_input_fn)
    best_exporter = tf.estimator.BestExporter(
                  name="best_exporter",  # Saved models are exported under /export/best_exporter/
                  compare_fn=_accuracy_bigger,
                  event_file_pattern='eval_{}/*.tfevents.*'.format(datetime.datetime.utcnow().strftime("%H%M%S")),
                  serving_input_receiver_fn=serving_input_fn,
                  exports_to_keep=5 )
    exporters = [Final_exporter]
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset("eval.csv", mode = tf.estimator.ModeKeys.EVAL),
        steps = None,
        start_delay_secs = 60, # start evaluating after N seconds
        throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
        exporters = exporters)
    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


In [94]:
output_dir = "babyweight_trained_LinearRegressor"
!rm -rf {output_dir}
train_and_evaluate_LinearRegressor(output_dir)

<<<<<<class 'list'>>>>>><<<<<[VocabularyListCategoricalColumn(key='is_male', vocabulary_list=('True', 'False', 'Unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='plurality', vocabulary_list=('Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)', 'Multiple(2+)'), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='mother_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='gestation_weeks', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='dummy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]>>>>>
INFO:tensorflow:Using config: {'_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4aff3ade80>, '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_device_fn': None, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_

In [99]:
def train_and_evaluate_DNNRegressor(output_dir):
    feature_columns, wide, deep = get_feature_cols()
        
    run_config = tf.estimator.RunConfig(tf_random_seed = 1)
    
    estimator = tf.estimator.DNNRegressor(
        feature_columns = deep,
        model_dir = output_dir,
        hidden_units = [128, 64, 32],
        config = run_config)
    

    
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset("train.csv", mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = TRAIN_STEPS)
    
    Last_exporter = tf.estimator.LatestExporter(name = "exporter", serving_input_receiver_fn = serving_input_fn)
    Final_exporter = tf.estimator.FinalExporter('./exporter', serving_input_receiver_fn=serving_input_fn)
    best_exporter = tf.estimator.BestExporter(
                  name="best_exporter",  # Saved models are exported under /export/best_exporter/
                  compare_fn=_accuracy_bigger,
                  event_file_pattern='eval_{}/*.tfevents.*'.format(datetime.datetime.utcnow().strftime("%H%M%S")),
                  serving_input_receiver_fn=serving_input_fn,
                  exports_to_keep=5 )
    exporters = [Final_exporter]
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset("eval.csv", mode = tf.estimator.ModeKeys.EVAL),
        steps = None,
        start_delay_secs = 60, # start evaluating after N seconds
        throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
        exporters = exporters)
    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


In [100]:
output_dir = "babyweight_trained_DNNRegressor"
!rm -rf {output_dir}
train_and_evaluate_DNNRegressor(output_dir)

INFO:tensorflow:Using config: {'_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4aff532390>, '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_device_fn': None, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_task_id': 0, '_is_chief': True, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_experimental_distribute': None, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_master': '', '_session_creation_timeout_secs': 7200, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_experimental_max_worker_delay_secs': None, '_model_dir': 'babyweight_trained_DNNRegressor', '_tf_random_seed': 1, '_train_distribute': None, '_global_id_in_cluster': 0, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_service': None, '_log_step_count_steps': 100}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training